In [1]:
import pickle
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import time
import numpy as np


In [2]:
df = pickle.load(open('Clayton_SG_FullPost_df.pkl', 'br'))


In [3]:
df.head()

,groups,long_post,post_number
0,[Acne],npAfter moving to a new country my skin began ...,396670
1,[PTSD],npIt happened again I went back in time becaus...,160394
2,"[Abuse, Codependency, Narcissist]",npThere is something that is really bothering ...,620850
3,[Military Family],npNo legal support from military after I caugh...,104230
4,[Narcissist],npI told him the narcissist that I was moving ...,621354


In [4]:
df.long_post[15642].split()[0]

'npMy'

In [5]:
# Create col of first pords to identify bad post

def bad_post(col = df['long_post']):
    fw = []
    b = col
    for i in range (0, len(b)):
        wrd = str(b[i]).split()[0]
        fw.append(wrd)
    return fw

df['first_word'] = bad_post()
print(df.shape)
df.head()



(34005, 4)


,groups,long_post,post_number,first_word
0,[Acne],npAfter moving to a new country my skin began ...,396670,npAfter
1,[PTSD],npIt happened again I went back in time becaus...,160394,npIt
2,"[Abuse, Codependency, Narcissist]",npThere is something that is really bothering ...,620850,npThere
3,[Military Family],npNo legal support from military after I caugh...,104230,npNo
4,[Narcissist],npI told him the narcissist that I was moving ...,621354,npI


In [6]:
#  drop bad post and col for filtering

df = df[df['first_word'] != 'nspanndivnspan']
df = df.drop(columns = ['first_word'])
print(df.shape)
df.head()

(34005, 3)


,groups,long_post,post_number
0,[Acne],npAfter moving to a new country my skin began ...,396670
1,[PTSD],npIt happened again I went back in time becaus...,160394
2,"[Abuse, Codependency, Narcissist]",npThere is something that is really bothering ...,620850
3,[Military Family],npNo legal support from military after I caugh...,104230
4,[Narcissist],npI told him the narcissist that I was moving ...,621354


In [7]:
df.isna().sum()

groups         0
long_post      0
post_number    0
dtype: int64

In [8]:
df = df.dropna()
df.isna().sum()

groups         0
long_post      0
post_number    0
dtype: int64

In [9]:
df = df.reset_index(drop=True)
print (df.shape)
df.head()

(34005, 3)


,groups,long_post,post_number
0,[Acne],npAfter moving to a new country my skin began ...,396670
1,[PTSD],npIt happened again I went back in time becaus...,160394
2,"[Abuse, Codependency, Narcissist]",npThere is something that is really bothering ...,620850
3,[Military Family],npNo legal support from military after I caugh...,104230
4,[Narcissist],npI told him the narcissist that I was moving ...,621354


In [10]:
len(df['long_post'])

34005

In [11]:
def np_gone(col = df['long_post']):
    nnp = []
    for i in range (0, len(col)):
        if col[i][0:2] == 'np':
            nnp.append(col[i][2:])
        else:
            nnp.append(col[i])
    return nnp

df['long_post'] = np_gone()
df.sample(5)


,groups,long_post,post_number
14498,[Adoption and Fostering],I really struggle with my daughters birth pare...,365496
3923,[PMS],Today has been an AWFUL day I always feel like...,55860
24671,[Klonopin],Who can relate to the fact your family says th...,107106
18526,[Online Dating],I dont think I would invest in getting into on...,125788
26177,"[Binge Eating, Obesity, Sugar Addiction]",I have gained back the lbs i lost last year C...,389974


In [12]:
import nltk
from nltk.tokenize import word_tokenize # Word Tokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

def tok_n_clean (txt):
    stop_words = stopwords.words('english')
    word_tokens = word_tokenize(txt)
    lemmatizer = WordNetLemmatizer()
    lowercase_tokens = [w.lower() for w in word_tokens]
    words = [word for word in lowercase_tokens if word.isalpha()]
    words = [w for w in words if not w in stop_words]
    words = [i for i in words if len(i) > 1]
    return words

def column_tok (col):
    lst= col.tolist()
    tok_lst = []
    for i in range (0, len(lst)):
        tok_lst.append(tok_n_clean(lst[i]))
    return tok_lst

df['long_post'] = column_tok (df['long_post'])

df.sample(5)



,groups,long_post,post_number
32338,"[Binge Eating, Depression, Emotional Abuse]","[bought, journal, today, sole, purpose, writin...",160683
20350,[Cervical Cancer],"[well, radiation, treatments, first, set, chem...",68725
17800,"[Anxiety, Chronic Fatigue Syndrome, Chronic Pain]","[need, hug]",355742
31004,[Anemia],"[animia, felt, realy, ill, energy, time, realy...",90256
14782,[Skin Picking],"[hard, lately, trying, stop, picking, sometime...",385436


In [13]:
df['groups'][2][1]

'Codependency'

In [14]:
# df with only first group of multi groups

df_first = df.copy()

def first_only (col = df_first['groups']):
    lst = []
    for i in range(0, len(col)):
        lst.append(col[i][0])
    return lst
df_first['groups'] = first_only()

print (df_first.shape)
df_first.sample(10)

(34005, 3)


,groups,long_post,post_number
23221,Brain Injury,"[god, please, let, sleep, stop, afraid, stop, ...",164044
26785,Narcissist,"[got, injunction]",622205
24389,Depression,"[posting, friend]",160863
7905,Hypothyroidism,"[hi, friends, im, new, diagnosed, hoshimotos, ...",577899
31057,Weight Loss,"[well, today, day, renew, vows, eat, better, t...",157149
19749,Dizziness,"[heard, certainly, intake, water, experience, ...",47562
19525,Parents,"[last, sunday, evening, fiancee, went, go, mee...",153529
7090,Jealousy,"[new, day, tactic, jealously, suspect, gf, som...",157279
33221,Schizophrenia,"[found, friend, illness, support]",60679
11525,Borderline Personality Disorder,"[trigger, warning]",619637


In [15]:
# df with nothing dropped post in multi groups are treated as separate

df_all = df.copy()
grp = pd.DataFrame(df_all.groups.values.tolist())
df_all['group1'] = grp[0]
df_all['group2'] = grp[1]
df_all['group3'] = grp[2]

df_all.sample(10)


,groups,long_post,post_number,group1,group2,group3
19101,[Alcohol],"[find, tht, get, lost, find, reachign, beer, o...",158949,Alcohol,None,None
10179,[Moms],"[dont, know, son, anxiety, depressionsince, di...",403309,Moms,None,None
14353,[Vicodin],"[need, help, quitting, vicodin, started, takin...",86863,Vicodin,None,None
23217,[Teen],"[im, mess, todaybrnfirst, period, one, prizes,...",606596,Teen,None,None
28285,[Kidney Cancer],"[good, evening, help, support, thise, kidney, ...",204903,Kidney Cancer,None,None
29618,"[Bisexuality, Coming Out, Gay and Lesbian]","[lgbtfocused, posti, point, feel, minoritized,...",400453,Bisexuality,Coming Out,Gay and Lesbian
15893,"[Alzheimers, Caregivers, Depression]","[recently, saw, houses, still, holiday, lights...",351284,Alzheimers,Caregivers,Depression
29383,[Learning Disability],"[ive, gotten, self, business, online, recently...",593407,Learning Disability,None,None
25368,[Single Dads],"[im, mike, im, year, old, man, trying, raise, ...",377881,Single Dads,None,None
31708,[Hair Loss],"[heres, article, covering, best, regrowth, tre...",39440,Hair Loss,None,None


In [16]:
df_drop = df_all.copy()

In [17]:
df_all1 = df_all[['post_number', 'long_post', 'group1']].copy()
df_all2 = df_all[['post_number', 'long_post', 'group2']].copy()
df_all3 = df_all[['post_number', 'long_post', 'group3']].copy()

In [18]:
df_all2 = df_all2.dropna()
df_all3 = df_all3.dropna()
df_all1 = df_all1.rename(columns={'group1': 'group'})
df_all2 = df_all2.rename(columns={'group2': 'group'})
df_all3 = df_all3.rename(columns={'group3': 'group'})

df_all = pd.concat([df_all1, df_all2, df_all3])
df_all = df_all.reindex()
print(df_all.shape)
df_all.sample(10)


(50368, 3)


,post_number,long_post,group
25188,51366,"[good, morning, one, good, morning, time, wake...",Non Hodgkins Lymphoma
13560,117384,"[lines, worst, enemy, anyone, else, trouble, w...",Irritable Bowel Syndrome
12774,59927,"[hello, allbrnive, forum, couple, weeks, ive, ...",Anxiety
26420,405466,"[never, drink, around, kids, moms, seems, like...",Alcohol
11919,371505,"[hello, group, im, new, group, syndrome, wante...",Chronic Fatigue Syndrome
19612,56830,"[experimenting, nicotine, patches, found, exce...",ADHD
14160,134058,"[hi, im, month, ive, suspected, months, parent...",Premature Ovarian Failure
6760,396677,"[someone, threatened, something, could, destro...",Bullying
10713,57268,[hello],Sexual Abuse
9025,430550,"[happy, valentines, day, wishes, go, wonderful...",Lymphedema


In [19]:
df_drop = df_drop[pd.isnull(df_drop['group2'])]
df_drop = df_drop.drop(columns = ['groups', 'group2', 'group3'])
df_all3 = df_all3.rename(columns={'group1': 'groups'})
print(df_drop.shape)
df_drop.sample(10)

(24429, 3)


,long_post,post_number,group1
11873,"[someone, spouse, partner, disrespects, money,...",375060,Financial Problems
24922,"[roof, anxiety, wont, even, let, enjoy, day, a...",606132,Panic Attack
32417,"[much, drama, tired, girl, school, bullying, s...",412777,Bullying
24685,"[worst, day, life, cant, get, life, folks, are...",152530,Sex Addiction
16938,"[chronic, pain, anxiety, fibromyalgia, ptsd, r...",97544,Seasonal Affective Disorder
6421,"[confused, sad, scared]",284405,Bone Cancer
25406,"[hi, hope, fine, yet, another, day, still, yes...",575465,PCOS
16562,"[diagnosed, couple, weeks, ago, pretty, sure, ...",413102,Herpes
9425,"[ok, discuss, issues, regarding, bondage, domi...",148199,Healthy Sex
20280,"[latelty, things, going, hill, life, marriage,...",60419,Suicide


In [20]:
# df dropping multi group post

